In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/face_mask_dataset_backup/train'
val_dir = '/content/drive/MyDrive/face_mask_dataset_backup/val'


In [ ]:
import os

print("Train classes:", os.listdir(train_dir))
print("Val classes:", os.listdir(val_dir))


Train classes: ['with_mask', 'without_mask']
Val classes: ['with_mask', 'without_mask']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam



In [ ]:
print(train_data.class_indices)
print(val_data.class_indices)


{'with_mask': 0, 'without_mask': 1}
{'with_mask': 0, 'without_mask': 1}


In [ ]:
import os

print("Train folder structure:")
for cls in os.listdir(train_dir):
    path = os.path.join(train_dir, cls)
    print(cls, ":", len(os.listdir(path)), "images")

print("\nValidation folder structure:")
for cls in os.listdir(val_dir):
    path = os.path.join(val_dir, cls)
    print(cls, ":", len(os.listdir(path)), "images")


Train folder structure:
with_mask : 1120 images
without_mask : 72375 images

Validation folder structure:
with_mask : 279 images
without_mask : 18093 images


In [ ]:
!ls /content


dataset_balanced  drive  sample_data


In [ ]:
!ls /content/dataset_balanced


train  val


In [ ]:
!ls /content/drive/MyDrive


 1000144273.jpg
 1456310679E-textChap-1Mod-3.pdf
 16_10_24.gdoc
 22_10_24.gdoc
 2245023265AE4CF87D02C8B6BA991139
 3f4a6d99-dd6d-4227-a234-b3a9eade54c5_1719583073%20-%2099.00%20To%20Aman%20%20Kesarwani%20on%20Google%20Pay.png
 Adaboost_Lab.ipynb
'arijit sanam re.mp3'
 assignment4.txt.gz
'Aswani krishna v'
'Aswani Krishna V (1).pdf'
'ASWANI KRISHNA V (1).pdf'
'ASWANI KRISHNA V (3).gdoc'
'ASWANI KRISHNA V (3).pdf'
'ASWANI KRISHNA V (4).pdf'
'ASWANI KRISHNA V (6).pdf'
'Aswani krishna v .Bsc mathematics.pdf'
'Aswani krishna V resume (1).pdf'
'Aswani-Krishna-V-Resume (1).pdf'
'Aswani krishna V resume.pdf'
'Aswani Krishna V_resume.pdf'
 Aswani-Krishna-V-Resume.pdf
'Aswani photo.jpeg'
'Aswani_Resume (1).pdf'
'Aswani_Resume (2).pdf'
'Aswani_Resume (3).pdf'
 Aswani_Resume.pdf
 AswaniSample2025Visit.docx
 AswaniSample2025Visit.gdoc
 AswaniSample2025Visit.pdf
'Aswani self declaration.gdoc'
 bandit.org.gdoc
 bandit.org.pdf
'Binsa self declaration.gdoc'
'certificate drug.jpg'
 Certificates.pdf
'cert

In [ ]:
!ls /content/drive/MyDrive/face_mask_dataset_backup


train  val


In [ ]:
import os, random, shutil


train_dir = '/content/drive/MyDrive/face_mask_dataset_backup/train'
val_dir = '/content/drive/MyDrive/face_mask_dataset_backup/val'

balanced_dir = '/content/drive/MyDrive/face_mask_dataset_balanced'
os.makedirs(os.path.join(balanced_dir, 'train/with_mask'), exist_ok=True)
os.makedirs(os.path.join(balanced_dir, 'train/without_mask'), exist_ok=True)
os.makedirs(os.path.join(balanced_dir, 'val/with_mask'), exist_ok=True)
os.makedirs(os.path.join(balanced_dir, 'val/without_mask'), exist_ok=True)

# Limits for balancing
limit_train = 1100
limit_val = 275

# Balance training set
for cls in ['with_mask', 'without_mask']:
    src_folder = os.path.join(train_dir, cls)
    dst_folder = os.path.join(balanced_dir, 'train', cls)
    imgs = os.listdir(src_folder)
    random.shuffle(imgs)
    for img in imgs[:limit_train]:
        shutil.copy(os.path.join(src_folder, img), os.path.join(dst_folder, img))

# Balance validation set
for cls in ['with_mask', 'without_mask']:
    src_folder = os.path.join(val_dir, cls)
    dst_folder = os.path.join(balanced_dir, 'val', cls)
    imgs = os.listdir(src_folder)
    random.shuffle(imgs)
    for img in imgs[:limit_val]:
        shutil.copy(os.path.join(src_folder, img), os.path.join(dst_folder, img))

print("✅ Balanced dataset created at:", balanced_dir)


✅ Balanced dataset created at: /content/drive/MyDrive/face_mask_dataset_balanced


In [ ]:
train_dir = '/content/drive/MyDrive/face_mask_dataset_balanced/train'
val_dir = '/content/drive/MyDrive/face_mask_dataset_balanced/val'


In [ ]:
import os

val_dir = '/content/drive/MyDrive/face_mask_dataset_balanced/val'

for cls in ['with_mask', 'without_mask']:
    path = os.path.join(val_dir, cls)
    print(cls, ":", len(os.listdir(path)), "images")


with_mask : 275 images
without_mask : 275 images


In [ ]:
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)


Found 550 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping


train_dir = '/content/drive/MyDrive/face_mask_dataset_balanced/train'
val_dir = '/content/drive/MyDrive/face_mask_dataset_balanced/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stop]
)



Found 2200 images belonging to 2 classes.
Found 550 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.7853 - loss: 0.4514 - val_accuracy: 0.9745 - val_loss: 0.1200
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9618 - loss: 0.1224 - val_accuracy: 0.9764 - val_loss: 0.0799
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.9734 - loss: 0.0796 - val_accuracy: 0.9782 - val_loss: 0.0684
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.9749 - loss: 0.0732 - val_accuracy: 0.9745 - val_loss: 0.0689
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.9860 - loss: 0.0524 - val_accuracy: 0.9727 - val_loss: 0.0706
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.9829 - loss: 0.0520 - val_accuracy: 0.9818 - val_loss: 0.0590
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.9849 - loss: 0.0406 - val_accuracy: 0.9727 - val_loss: 0.0638
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9842 - loss: 0.0412 - val_accuracy: 0.9836 - v

In [ ]:
model.save('/content/drive/MyDrive/face_mask_model_balanced.h5')
print("✅ Model trained and saved successfully!")

✅ Model trained and saved successfully!
